# Mancala Game

In [5]:
#se crean las casillas mancala la lista de tablero
mancala = [0,0]

board = [0]*2
for i in range(2):
    board[i] = [4]*6



In [8]:
#objeto tablero
    # tiene un tablero y mancalas
    # metodo para realizar jugada de P1 o P2
    # metodo para saber si fila x == 0

class Tablero:
    def __init__(self):
        #inicializa el tablero y mancala
        mancala = [0,0]
        board = [0]*2
        for i in range(2):
            board[i] = [4]*6
    
    
    #devuelve True si la fila seleccionada (i [0,1]) está vacía
    def filaVacia(self,i):
        for j in range(8):
            if(board[i][j] != 0):
                return False
        return True

In [ ]:
#objeto IA
    # tiene metodo para realizar turno
    # tiene metodo para realizar monte carlo


In [ ]:
#Game
    # decide a quien darle el turno
    # tiene el objeto IA
    # tiene la lógica del jugador humano

In [10]:
#test block
game = Tablero()

print(game.filaVacia(1))

False
